In [2]:
X_He=0.941

In [3]:
mw_Air=28.9660
mw_He=4.0026
gamma_Air=1.4020
gamma_He=1.6670

R=8314.4598
R_Air=R/mw_Air
R_He=R/mw_He


cp_Air=gamma_Air/(gamma_Air-1)*R_Air
cp_He=gamma_He/(gamma_He-1)*R_He

cv_Air=R_Air/(gamma_Air-1)
cv_He=R_He/(gamma_He-1)


mw_test=X_He*mw_He+(1-X_He)*mw_Air
Y_He=(X_He*mw_He)/(mw)

cp=cp_Air*(1-Y_He)+cp_He*Y_He
cv=cv_Air*(1-Y_He)+cv_He*Y_He
gamma_test=(cp)/(cv)


NameError: name 'mw' is not defined

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model

def calc_shock(m, p1, t1, drivengas, drivergas, t4, areafactor,a=mw_test,b=gamma_test):
    # Define constants
    ru = 8314.51  # Universal gas constant (J/kmol·K)

    # Define gas properties for driven gas
    driven_gas_properties = {
        "1": {"g": 1.4020, "mw": 28.9660},   # Air
        "2": {"g": 1.6700, "mw": 39.9480},   # Argon
        "3": {"g": 1.2970, "mw": 44.0100},   # Carbon Dioxide
        "4": {"g": 1.6670, "mw": 4.0026},    # Helium
        "5": {"g": 1.6670, "mw": 20.1797},   # Neon
        "6": {"g": 1.4010, "mw": 28.0134},   # Nitrogen
        "7": {"g": 1.6670, "mw": 131.2900},  # Xenon
        "8": {"g": 1.0920, "mw": 146.0600},  # Sulfur Hexafluoride
        "9": {"g": 1.6680, "mw": 21.9800},   # Mix: He/Ar 50/50
        "10": {"g": b, "mw": a} 
    }

    # Define gas properties for driver gas
    driver_gas_properties = driven_gas_properties.copy()

    g = driven_gas_properties[drivengas]["g"]
    mw = driven_gas_properties[drivengas]["mw"]
    g4 = driver_gas_properties[drivergas]["g"]
    mw4 = driver_gas_properties[drivergas]["mw"]

    # Calculate additional parameters
    r = ru / mw
    r4 = ru / mw4
    gp = g + 1.0
    gm = g - 1.0
    a1 = math.sqrt(g * r * t1)
    w = a1 * m
    d1 = p1 / (r * t1)

    # State 2 calculations
    p2p1 = 1 + (2 * g / gp) * (m**2 - 1)
    p2 = p2p1 * p1
    t2t1 = p2p1 * ((gp / gm + p2p1) / (1 + gp / gm * p2p1))
    t2 = t2t1 * t1
    a2 = math.sqrt(g * r * t2)
    d2d1 = (1 + (gp / gm) * p2p1) / (gp / gm + p2p1)
    d2 = d2d1 * d1
    u2 = (a1 / g) * (p2p1 - 1) * math.sqrt((2 * g / gp) / (p2p1 + gm / gp))

    # State 4 calculations
    a4 = math.sqrt(g4 * r4 * t4)
    p4p1_arg = 1 - ((g4 - 1) * (a1 / a4) * (p2p1 - 1)) / math.sqrt(2 * g * (2 * g + gp * (p2p1 - 1)))
    p4p1_exp = -2 * g4 / (g4 - 1)
    p4p1 = p2p1 * (p4p1_arg**p4p1_exp)
    p4 = p4p1 * p1
    d4 = p4 / (r4 * t4)

    # State 5 calculations
    p5p2 = ((3 * g - 1) * p2p1 - gm) / (gm * p2p1 + gp)
    p5 = p5p2 * p2
    t5t2 = p5p2 * ((gp / gm + p5p2) / (1 + gp / gm * p5p2))
    t5 = t5t2 * t2
    d5d2 = (1 + (gp / gm) * p5p2) / (gp / gm + p5p2)
    d5 = d5d2 * d2
    a5 = math.sqrt(g * r * t5)

    # Output results
    results = {
        "State 1": {"Pressure (MPa)": p1 / 1e6, "Temperature (K)": t1, "Density (kg/m^3)": d1, "Sound Speed (m/s)": a1},
        "State 2": {"Pressure (MPa)": p2 / 1e6, "Temperature (K)": t2, "Density (kg/m^3)": d2, "Sound Speed (m/s)": a2, "Velocity (m/s)": u2},
        "State 4": {"Pressure (MPa)": p4 / 1e6, "Temperature (K)": t4, "Density (kg/m^3)": d4, "Sound Speed (m/s)": a4},
        "State 5": {"Pressure (MPa)": p5 / 1e6, "Temperature (K)": t5, "Density (kg/m^3)": d5, "Sound Speed (m/s)": a5}
    }

    return results

NameError: name 'mw_test' is not defined

In [4]:
# Example usage
m = 3.365  # Mach number
p1 = 122199.35  # Initial pressure (Pa)
t1 = 293.41888  # Initial temperature (K)
drivengas = "1"  # Driven gas (Air)
drivergas = "10"  # Driver gas (Helium)
t4 = 301.706  # Initial temperature of driver section (K)
areafactor = 1.35

results = calc_shock(m, p1, t1, drivengas, drivergas, t4, areafactor)

for state, properties in results.items():
    print(f"{state}:")
    for prop, value in properties.items():
        print(f"  {prop}: {value}")

NameError: name 'calc_shock' is not defined

In [ ]:
# Define input ranges
m_values = np.linspace(1,12,551)
p1_values = 10**(np.linspace(2,6,401))
t1_value = 300
drivengas_values = ["1", "3"]
drivergas_values = ["4", "6"]
t4_value = 350

# Prepare results list
results_list = []

# Iterate over all combinations of inputs
for m in m_values:
    for p1 in p1_values:
        for drivengas in drivengas_values:
            for drivergas in drivergas_values:
                try:
                    results = calc_shock(m, p1, t1_value, drivengas, drivergas, t4_value, areafactor)
                    if all(results["State 1"].values()) and all(results["State 2"].values()) and all(results["State 4"].values()) and all(results["State 5"].values()):
                        results_list.append({
                            "m": m,
                            "p1 (Pa)": p1,
                            "t1 (K)": t1_value,
                            "drivengas": drivengas,
                            "drivergas": drivergas,
                            "t4 (K)": t4_value,
                            **{f"State 1 {key}": val for key, val in results["State 1"].items()},
                            **{f"State 2 {key}": val for key, val in results["State 2"].items()},
                            **{f"State 4 {key}": val for key, val in results["State 4"].items()},
                            **{f"State 5 {key}": val for key, val in results["State 5"].items()},
                        })
                except Exception as e:
                    # Skip cases with errors or issues
                    pass
    print(m)
# Convert results to a DataFrame
df = pd.DataFrame(results_list)

# Remove rows with any empty values
df.dropna(inplace=True)

# Save to Excel
# df.to_excel("shock_results.xlsx", index=False)

print("Excel file 'shock_results.xlsx' has been created.")


/var/folders/tn/51vr96bd29l35lz788b3jxym0000gn/T/ipykernel_2023/1353928662.py:55: RuntimeWarning: invalid value encountered in double_scalars
  p4p1 = p2p1 * (p4p1_arg**p4p1_exp)


Excel file 'shock_results.xlsx' has been created.


In [60]:
df.keys()

Index(['m', 'p1 (Pa)', 't1 (K)', 'drivengas', 'drivergas', 't4 (K)',
       'State 1 Pressure (MPa)', 'State 1 Temperature (K)',
       'State 1 Density (kg/m^3)', 'State 1 Sound Speed (m/s)',
       'State 2 Pressure (MPa)', 'State 2 Temperature (K)',
       'State 2 Density (kg/m^3)', 'State 2 Sound Speed (m/s)',
       'State 2 Velocity (m/s)', 'State 4 Pressure (MPa)',
       'State 4 Temperature (K)', 'State 4 Density (kg/m^3)',
       'State 4 Sound Speed (m/s)', 'State 5 Pressure (MPa)',
       'State 5 Temperature (K)', 'State 5 Density (kg/m^3)',
       'State 5 Sound Speed (m/s)'],
      dtype='object')

In [61]:
# Load the data from the generated Excel file
data = pd.read_excel("shock_results.xlsx")

# Select input and output features
X = data[["p1 (Pa)", "t1 (K)", "drivengas", "drivergas",'State 4 Pressure (MPa)', "t4 (K)"]]
y = data["m"]

# Preprocess the categorical features
drivengas_mapping = {str(val): idx for idx, val in enumerate(drivengas_values)}
drivergas_mapping = {str(val): idx for idx, val in enumerate(drivergas_values)}
X["drivengas"] = X["drivengas"].map(drivengas_mapping)
X["drivergas"] = X["drivergas"].map(drivergas_mapping)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input data
X_train = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test = (X_test - X_test.min()) / (X_test.max() - X_test.min())

# Build the CNN model
model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train the model
model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
eval_results = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {eval_results[0]}, Test MAE: {eval_results[1]}")

# Save the trained model
model.save("shock_model.h5")
print("Trained model saved as 'shock_model.h5'.")


/var/folders/tn/51vr96bd29l35lz788b3jxym0000gn/T/ipykernel_2023/786452612.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["drivengas"] = X["drivengas"].map(drivengas_mapping)
/var/folders/tn/51vr96bd29l35lz788b3jxym0000gn/T/ipykernel_2023/786452612.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["drivergas"] = X["drivergas"].map(drivergas_mapping)


Epoch 1/50
    8/13787 [..............................] - ETA: 1:42 - loss: 40.7348 - mae: 5.6849 

2024-12-13 16:55:46.550392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13782/13787 [============================>.] - ETA: 0s - loss: 13.3462 - mae: 2.9154

2024-12-13 16:57:09.855162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13787/13787 [==============================] - 93s 7ms/step - loss: 13.3446 - mae: 2.9152 - val_loss: 8.4548 - val_mae: 2.4402
Epoch 2/50
13787/13787 [==============================] - 93s 7ms/step - loss: 8.4732 - mae: 2.4437 - val_loss: 8.4543 - val_mae: 2.4413
Epoch 3/50
13787/13787 [==============================] - 93s 7ms/step - loss: 8.4730 - mae: 2.4437 - val_loss: 8.4543 - val_mae: 2.4418
Epoch 4/50
 4489/13787 [========>.....................] - ETA: 58s - loss: 8.4755 - mae: 2.4449

KeyboardInterrupt: 

In [62]:
# Evaluate the model
eval_results = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {eval_results[0]}, Test MAE: {eval_results[1]}")

# Save the trained model
model.save("shock_model.h5")
print("Trained model saved as 'shock_model.h5'.")

Test Loss: 8.460789680480957, Test MAE: 2.4422335624694824
Trained model saved as 'shock_model.h5'.


In [ ]:


# Load the trained model
model = load_model("shock_model.h5")

# Example input data for prediction
example_input = {
    "p1 (Pa)": 5000,
    "t1 (K)": 300,
    "drivengas": "1",
    "drivergas": "4",
    'State 4 Pressure (MPa)': 12,
    "t4 (K)": 350
}

# Preprocess the input data
drivengas_mapping = {"1": 0, "3": 1}
drivergas_mapping = {"4": 0, "6": 1}

example_input["drivengas"] = drivengas_mapping[example_input["drivengas"]]
example_input["drivergas"] = drivergas_mapping[example_input["drivergas"]]

input_data = pd.DataFrame([example_input])
input_data = (input_data - input_data.min()) / (input_data.max() - input_data.min())

# Predict the value of m
predicted_m = model.predict(input_data)
print(f"Predicted m: {predicted_m[0][0]}")


Predicted m: 5.579399108886719


2024-12-13 17:02:07.035375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
